# Mejorando mi red convolucional

In [1]:
import os
import random
import numpy as np
import tensorflow as tf

# De esta manera se ajusta la semilla para que sea replicable
seed_value= 448

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
from keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np

import os

# Establecer el número deseado de núcleos
os.environ["TF_NUM_THREADS"] = "10"

(train_X, train_Y),(test_X, test_Y) = mnist.load_data()

clases = 10
dimensiones = (28, 28, 1)

train_X = train_X.astype("float32") / 255
test_X = test_X.astype("float32") / 255

train_X = np.expand_dims(train_X, -1)
test_X = np.expand_dims(test_X, -1)

# convert class vectors to binary class matrices
train_Y = to_categorical(train_Y, clases)
test_Y = to_categorical(test_Y, clases)

11490434/11490434 [==============================] - 0s 0us/step


### Funcion para crear y compilar el modelo

In [3]:
def create_model(lay_Filters=2, kernel_size=(2,2), pool_size=(2,2), cUnits=4, units=200, rate=0.5):
    model = keras.Sequential()

    # añadimos las capas
    model.add(keras.Input(shape=dimensiones))

    # da errores si es mayor que 4
    if lay_Filters > 4:
        lay_Filters=4
    filters = 32

    # capas convolucionales y de agregacion que queramos
    for i in range(lay_Filters):
        model.add(layers.Conv2D(filters, kernel_size=kernel_size, activation="relu", padding="same"))
        model.add(layers.MaxPool2D(pool_size=pool_size))
        filters *= 2

    # capa que aplana
    model.add(layers.Flatten())

    # capa que ayuda a reducir el sobreajuste
    model.add(layers.Dropout(rate=rate))

    # for con las capas de neuronas que queramos añadir
    for i in range(cUnits):
        model.add(layers.Dense(units, activation="relu"))

     # capa de salida, el nº de nueronas tiene que coincidir con el nº de clases
    model.add(layers.Dense(clases, activation="softmax"))



    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

### Configuración de la validación cruzada y búsqueda de hiperparámetros

In [5]:
# %pip install scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold, GridSearchCV

redes = KerasClassifier(
    model=create_model
)

hypers1 = dict(
    model__kernel_size=[(2,2),(3,3)],
    model__rate=[0.5,0.75],
    epochs=[10,30],
    batch_size=[512,256]
)

crossV = KFold(
    n_splits=3,
    shuffle=True,
    random_state=448
)

search = GridSearchCV(
    redes,
    param_grid=hypers1,
    cv=crossV,
    verbose=10
)


## Ejecución

In [6]:
result = search.fit(train_X, train_Y)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3; 1/16] START batch_size=512, epochs=10, model__kernel_size=(2, 2), model__rate=0.5
Epoch 1/10
79/79 [==============================] - 17s 13ms/step - loss: 0.6605 - accuracy: 0.7889
Epoch 2/10
79/79 [==============================] - 1s 11ms/step - loss: 0.1651 - accuracy: 0.9496
Epoch 3/10
79/79 [==============================] - 1s 12ms/step - loss: 0.1084 - accuracy: 0.9665
Epoch 4/10
79/79 [==============================] - 1s 12ms/step - loss: 0.0884 - accuracy: 0.9718
Epoch 5/10
79/79 [==============================] - 1s 12ms/step - loss: 0.0725 - accuracy: 0.9766
Epoch 6/10
79/79 [==============================] - 1s 11ms/step - loss: 0.0647 - accuracy: 0.9799
Epoch 7/10
79/79 [==============================] - 1s 11ms/step - loss: 0.0535 - accuracy: 0.9829
Epoch 8/10
79/79 [==============================] - 1s 10ms/step - loss: 0.0476 - accuracy: 0.9849
Epoch 9/10
79/79 [==============================] - 1s 

Resultados de la experimentación con hiperparámetros

In [14]:
import pandas as pd

# resultados del gridsearch

result = pd.DataFrame(search.cv_results_)
result = result.sort_values(by=["rank_test_score"])
result = result.set_index(
    result["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("setting")
result[["params", "mean_test_score", "std_test_score"]]

,params,mean_test_score,std_test_score
setting,,,
"512_30_(3, 3)_0.75","{'batch_size': 512, 'epochs': 30, 'model__kern...",0.991983,0.000347
"256_30_(3, 3)_0.75","{'batch_size': 256, 'epochs': 30, 'model__kern...",0.991817,0.000760
"512_30_(3, 3)_0.5","{'batch_size': 512, 'epochs': 30, 'model__kern...",0.991267,0.000664
"256_30_(3, 3)_0.5","{'batch_size': 256, 'epochs': 30, 'model__kern...",0.991267,0.000397
"256_30_(2, 2)_0.75","{'batch_size': 256, 'epochs': 30, 'model__kern...",0.990683,0.000566
"512_30_(2, 2)_0.75","{'batch_size': 512, 'epochs': 30, 'model__kern...",0.990633,0.000386
"256_30_(2, 2)_0.5","{'batch_size': 256, 'epochs': 30, 'model__kern...",0.990600,0.000698
"512_30_(2, 2)_0.5","{'batch_size': 512, 'epochs': 30, 'model__kern...",0.990283,0.000409
"256_10_(3, 3)_0.5","{'batch_size': 256, 'epochs': 10, 'model__kern...",0.989733,0.001062


### Probamos otro gurpo de hiper parametros



In [17]:
redes2 = KerasClassifier(
    model=create_model
)

hypers2 = dict(
    model__kernel_size=[(3,3)],
    model__rate=[0.75],
    epochs=[30],
    batch_size=[512],
    model__lay_Filters=[3,4],
    model__cUnits=[4,8],
    model__units=[400,800]

)

crossV = KFold(
    n_splits=3,
    shuffle=True,
    random_state=448
)

search = GridSearchCV(
    redes2,
    param_grid=hypers2,
    cv=crossV,
    verbose=10
)

In [18]:
result2 = search.fit(train_X, train_Y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3; 1/8] START batch_size=512, epochs=30, model__cUnits=4, model__kernel_size=(3, 3), model__lay_Filters=3, model__rate=0.75, model__units=400
Epoch 1/30
79/79 [==============================] - 4s 16ms/step - loss: 0.8049 - accuracy: 0.7206
Epoch 2/30
79/79 [==============================] - 1s 15ms/step - loss: 0.1782 - accuracy: 0.9433
Epoch 3/30
79/79 [==============================] - 1s 15ms/step - loss: 0.1277 - accuracy: 0.9590
Epoch 4/30
79/79 [==============================] - 1s 16ms/step - loss: 0.0995 - accuracy: 0.9690
Epoch 5/30
79/79 [==============================] - 1s 17ms/step - loss: 0.0856 - accuracy: 0.9735
Epoch 6/30
79/79 [==============================] - 1s 16ms/step - loss: 0.0790 - accuracy: 0.9751
Epoch 7/30
79/79 [==============================] - 1s 18ms/step - loss: 0.0713 - accuracy: 0.9777
Epoch 8/30
79/79 [==============================] - 1s 15ms/step - loss: 0.0621 - accuracy: 0.9808


### Imprimir

In [19]:
# resultados del gridsearch

result2 = pd.DataFrame(search.cv_results_)
result2 = result2.sort_values(by=["rank_test_score"])
result2 = result2.set_index(
    result2["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("setting")
result2[["params", "mean_test_score", "std_test_score"]]

,params,mean_test_score,std_test_score
setting,,,
"512_30_4_(3, 3)_3_0.75_800","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.993317,0.000239
"512_30_8_(3, 3)_3_0.75_400","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.992833,0.000595
"512_30_8_(3, 3)_3_0.75_800","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.992433,0.000103
"512_30_4_(3, 3)_3_0.75_400","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.991833,0.000811
"512_30_4_(3, 3)_4_0.75_800","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.991300,0.000248
"512_30_8_(3, 3)_4_0.75_400","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.990300,0.001553
"512_30_4_(3, 3)_4_0.75_400","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.989750,0.001677
"512_30_8_(3, 3)_4_0.75_800","{'batch_size': 512, 'epochs': 30, 'model__cUni...",0.952433,0.045808
